In [1]:
import gym
import numpy as np
from collections import deque
import matplotlib.pyplot as plt

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

Using TensorFlow backend.
/home/irfan_hasib/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/irfan_hasib/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/irfan_hasib/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/irfan_hasib/.local/lib/python3.6/site-packag

In [3]:
class MountainCar():
    
    def __init__(self):
        self.env = gym.make('MountainCar-v0') 
        self.action_shape = (self.env.action_space.n,)
        self.state_shape = self.env.observation_space.shape
        #print("Env Space : ", self.state_shape)
        #print("Action Space : ", self.action_shape)
        self.test_mode = False
    
    def reset(self):
        return self.env.reset()
    
    def sample(self):
        return self.env.action_space.sample()
    
    def step(self, action):
        state, reward, terminal, _ = self.env.step(int(action))
        #reward = 0
        
        #if terminal:
            #if state[0] > 0.5:
                #reward = 1
            #else:
                #reward = state[0] + 0.5
         
        return state, reward, terminal
    
    def test_start(self):
        self.animation.reset()
        self.env.reset()
    
    def test_epoch_step(self):
        pass
        #self.animation.store(self.env.render(mode="rgb_array"))
    
    def test_close(self):
        #self.env.close() 
        self.env.viewer = None
                
MC = MountainCar()
st_shape=MC.state_shape[0]
act_shape=MC.action_shape[0]



In [4]:
model = Sequential()
model.add(Dense(100,activation='relu',input_shape=(st_shape,)))
model.add(Dense(100,activation='relu'))
model.add(Dense(act_shape,activation='linear'))
model.compile(loss='mse',optimizer=Adam(lr=0.001),)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               300       
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 303       
Total params: 10,703
Trainable params: 10,703
Non-trainable params: 0
_________________________________________________________________


In [5]:
class Memory():
    def __init__(self, max_size):
        self.buffer = deque(maxlen = max_size)
    
    def add(self, experience):
        self.buffer.append(experience)
    
    def sample(self, batch_size):
        buffer_size = len(self.buffer)
        index = np.random.choice(np.arange(buffer_size),
                                size = batch_size,
                                replace = False)
        
        return [self.buffer[i] for i in index]


In [6]:
memory_size=10000
state=MC.reset()
memory =Memory(memory_size)
for i in range(memory_size):
    action = np.random.randint(0,act_shape)
    new_state,reward,terminal=MC.step(action)
    experience=[state,action,reward,new_state,terminal]
    memory.add(experience)
    new_state=state
    
    
    

In [7]:
state.reshape(1,-1).shape
new_state,reward,terminal=MC.step(action)

In [8]:
ks=True
factor=[1,100]
batch_size=100
gamma=0.45
episodes=1000
epsilon=0.95
epsilon_decay=0.995
steps=200
lr=0.001

treminal=False

state=MC.reset()
state=state*factor
optimizer = Adam(lr = 0.001)
for ep in range(episodes):
    total_reward=0
    state=MC.reset()
    state=state*factor
    for step in range(steps):
        if np.random.random()< epsilon:
            action = np.random.randint(0,act_shape)
            #print('random',step,action,reward)
        else:
            #print('model',step,action,reward)
            if ks==True:
                Qvals=model.predict(state.reshape(1,-1))
            action=np.argmax(Qvals)
            #print(Qvals)
        
        new_state,reward,terminal=MC.step(action)
        #print(new_state,action,reward)
        new_state=new_state*factor
        total_reward+=reward
        if terminal==True or total_reward<-199:
            
            if total_reward<-199:
                reward =-100
                experience=[state,action,reward,new_state,terminal]
                memory.add(experience)
                print("Episode %d, Failed! Reward %d"%(ep,total_reward))
            elif total_reward<-110 and total_reward>-199:
                reward=10
                experience=[state,action,reward,new_state,terminal]
                memory.add(experience)
                print("Episode %d, Better! Reward %d"%(ep,total_reward))
            elif total_reward>=-110:
                reward=100
                experience=[state,action,reward,new_state,terminal]
                memory.add(experience)
                print("Episode %d, Passed! Reward %d"%(ep,total_reward))
            break
        
        experience=[state,action,reward,new_state,terminal]
        memory.add(experience)
        state=new_state
    
    batch=memory.sample(batch_size)
    states_ep= np.array([each[0] for each in batch], ndmin=3).reshape(len(batch),-1)
    actions_ep = np.array([each[1] for each in batch])#.reshape(len(batch),-1)
    rewards_ep = np.array([each[2] for each in batch])#.reshape(len(batch),-1)
    next_states_ep = np.array([each[3] for each in batch], ndmin=3).reshape(len(batch),-1)
    dones_ep = np.array([each[4] for each in batch])#.reshape(len(batch),-1)
  
    if ks==True:
        Qvals=model.predict(states_ep)
        new_Qvals=model.predict(next_states_ep)
    Calc_Qvals=rewards_ep+gamma*(np.max(new_Qvals,axis=1)*(~dones_ep))
    target_Qvals=np.array(Qvals.copy())
    
    #print(Qvals[0],target_Qvals[0],Calc_Qvals[0])
    for i in range(len(actions_ep)):
        target_Qvals[i,actions_ep[i]]=Calc_Qvals[i]
    #print(Qvals[0],target_Qvals[0],Calc_Qvals[0])
    
    if ks==True:
        print('training')
        model.fit(states_ep,target_Qvals,verbose=0)
    epsilon=epsilon*epsilon_decay
    print('episopde',ep,'epsilon',epsilon,'reward',total_reward)
    

Episode 0, Failed! Reward -200
training
episopde 0 epsilon 0.9452499999999999 reward -200.0
Episode 1, Failed! Reward -200
training
episopde 1 epsilon 0.9405237499999999 reward -200.0
Episode 2, Failed! Reward -200
training
episopde 2 epsilon 0.9358211312499999 reward -200.0
Episode 3, Failed! Reward -200
training
episopde 3 epsilon 0.9311420255937499 reward -200.0
Episode 4, Failed! Reward -200
training
episopde 4 epsilon 0.9264863154657812 reward -200.0
Episode 5, Failed! Reward -200
training
episopde 5 epsilon 0.9218538838884522 reward -200.0
Episode 6, Failed! Reward -200
training
episopde 6 epsilon 0.91724461446901 reward -200.0
Episode 7, Failed! Reward -200
training
episopde 7 epsilon 0.9126583913966649 reward -200.0
Episode 8, Failed! Reward -200
training
episopde 8 epsilon 0.9080950994396816 reward -200.0
Episode 9, Failed! Reward -200
training
episopde 9 epsilon 0.9035546239424832 reward -200.0
Episode 10, Failed! Reward -200
training
episopde 10 epsilon 0.8990368508227707 re

Episode 88, Failed! Reward -200
training
episopde 88 epsilon 0.6081039041197829 reward -200.0
Episode 89, Failed! Reward -200
training
episopde 89 epsilon 0.605063384599184 reward -200.0
Episode 90, Failed! Reward -200
training
episopde 90 epsilon 0.6020380676761881 reward -200.0
Episode 91, Failed! Reward -200
training
episopde 91 epsilon 0.5990278773378072 reward -200.0
Episode 92, Failed! Reward -200
training
episopde 92 epsilon 0.5960327379511181 reward -200.0
Episode 93, Failed! Reward -200
training
episopde 93 epsilon 0.5930525742613625 reward -200.0
Episode 94, Failed! Reward -200
training
episopde 94 epsilon 0.5900873113900557 reward -200.0
Episode 95, Failed! Reward -200
training
episopde 95 epsilon 0.5871368748331054 reward -200.0
Episode 96, Failed! Reward -200
training
episopde 96 epsilon 0.5842011904589398 reward -200.0
Episode 97, Failed! Reward -200
training
episopde 97 epsilon 0.5812801845066451 reward -200.0
Episode 98, Failed! Reward -200
training
episopde 98 epsilon 

Episode 174, Failed! Reward -200
training
episopde 174 epsilon 0.39515068195968583 reward -200.0
Episode 175, Failed! Reward -200
training
episopde 175 epsilon 0.3931749285498874 reward -200.0
Episode 176, Failed! Reward -200
training
episopde 176 epsilon 0.39120905390713795 reward -200.0
Episode 177, Failed! Reward -200
training
episopde 177 epsilon 0.38925300863760226 reward -200.0
Episode 178, Failed! Reward -200
training
episopde 178 epsilon 0.38730674359441425 reward -200.0
Episode 179, Failed! Reward -200
training
episopde 179 epsilon 0.38537020987644216 reward -200.0
Episode 180, Failed! Reward -200
training
episopde 180 epsilon 0.38344335882705993 reward -200.0
Episode 181, Failed! Reward -200
training
episopde 181 epsilon 0.3815261420329246 reward -200.0
Episode 182, Failed! Reward -200
training
episopde 182 epsilon 0.37961851132276 reward -200.0
Episode 183, Failed! Reward -200
training
episopde 183 epsilon 0.37772041876614615 reward -200.0
Episode 184, Failed! Reward -200
tr

Episode 259, Failed! Reward -200
training
episopde 259 epsilon 0.25806232111804345 reward -200.0
Episode 260, Failed! Reward -200
training
episopde 260 epsilon 0.2567720095124532 reward -200.0
Episode 261, Failed! Reward -200
training
episopde 261 epsilon 0.25548814946489096 reward -200.0
Episode 262, Failed! Reward -200
training
episopde 262 epsilon 0.2542107087175665 reward -200.0
Episode 263, Failed! Reward -200
training
episopde 263 epsilon 0.2529396551739787 reward -200.0
Episode 264, Failed! Reward -200
training
episopde 264 epsilon 0.2516749568981088 reward -200.0
Episode 265, Failed! Reward -200
training
episopde 265 epsilon 0.2504165821136183 reward -200.0
Episode 266, Better! Reward -187
training
episopde 266 epsilon 0.2491644992030502 reward -187.0
Episode 267, Better! Reward -161
training
episopde 267 epsilon 0.24791867670703494 reward -161.0
Episode 268, Better! Reward -151
training
episopde 268 epsilon 0.24667908332349975 reward -151.0
Episode 269, Better! Reward -168
tra

Episode 344, Failed! Reward -200
training
episopde 344 epsilon 0.16853358635384177 reward -200.0
Episode 345, Failed! Reward -200
training
episopde 345 epsilon 0.16769091842207257 reward -200.0
Episode 346, Failed! Reward -200
training
episopde 346 epsilon 0.1668524638299622 reward -200.0
Episode 347, Failed! Reward -200
training
episopde 347 epsilon 0.16601820151081237 reward -200.0
Episode 348, Failed! Reward -200
training
episopde 348 epsilon 0.1651881105032583 reward -200.0
Episode 349, Failed! Reward -200
training
episopde 349 epsilon 0.16436216995074202 reward -200.0
Episode 350, Failed! Reward -200
training
episopde 350 epsilon 0.1635403591009883 reward -200.0
Episode 351, Failed! Reward -200
training
episopde 351 epsilon 0.16272265730548335 reward -200.0
Episode 352, Failed! Reward -200
training
episopde 352 epsilon 0.16190904401895595 reward -200.0
Episode 353, Failed! Reward -200
training
episopde 353 epsilon 0.16109949879886118 reward -200.0
Episode 354, Failed! Reward -200


...

Episode 599, Better! Reward -122
training
episopde 599 epsilon 0.04694313100453668 reward -122.0
Episode 600, Better! Reward -124
training
episopde 600 epsilon 0.046708415349514 reward -124.0
Episode 601, Better! Reward -163
training
episopde 601 epsilon 0.04647487327276643 reward -163.0
Episode 602, Better! Reward -155
training
episopde 602 epsilon 0.04624249890640259 reward -155.0
Episode 603, Better! Reward -157
training
episopde 603 epsilon 0.04601128641187058 reward -157.0
Episode 604, Better! Reward -163
training
episopde 604 epsilon 0.045781229979811225 reward -163.0
Episode 605, Better! Reward -163
training
episopde 605 epsilon 0.045552323829912165 reward -163.0
Episode 606, Better! Reward -167
training
episopde 606 epsilon 0.04532456221076261 reward -167.0
Episode 607, Better! Reward -162
training
episopde 607 epsilon 0.0450979393997088 reward -162.0
Episode 608, Better! Reward -161
training
episopde 608 epsilon 0.04487244970271025 reward -161.0
Episode 609, Better! Reward -17

Episode 684, Failed! Reward -200
training
episopde 684 epsilon 0.030657300874442236 reward -200.0
Episode 685, Failed! Reward -200
training
episopde 685 epsilon 0.030504014370070026 reward -200.0
Episode 686, Failed! Reward -200
training
episopde 686 epsilon 0.030351494298219674 reward -200.0
Episode 687, Failed! Reward -200
training
episopde 687 epsilon 0.030199736826728576 reward -200.0
Episode 688, Failed! Reward -200
training
episopde 688 epsilon 0.030048738142594934 reward -200.0
Episode 689, Failed! Reward -200
training
episopde 689 epsilon 0.02989849445188196 reward -200.0
Episode 690, Failed! Reward -200
training
episopde 690 epsilon 0.02974900197962255 reward -200.0
Episode 691, Failed! Reward -200
training
episopde 691 epsilon 0.029600256969724438 reward -200.0
Episode 692, Failed! Reward -200
training
episopde 692 epsilon 0.029452255684875816 reward -200.0
Episode 693, Better! Reward -152
training
episopde 693 epsilon 0.029304994406451436 reward -152.0
Episode 694, Better! R

Episode 768, Better! Reward -124
training
episopde 768 epsilon 0.020122072005079417 reward -124.0
Episode 769, Better! Reward -119
training
episopde 769 epsilon 0.02002146164505402 reward -119.0
Episode 770, Failed! Reward -200
training
episopde 770 epsilon 0.01992135433682875 reward -200.0
Episode 771, Better! Reward -118
training
episopde 771 epsilon 0.01982174756514461 reward -118.0
Episode 772, Better! Reward -124
training
episopde 772 epsilon 0.019722638827318884 reward -124.0
Episode 773, Better! Reward -118
training
episopde 773 epsilon 0.01962402563318229 reward -118.0
Episode 774, Better! Reward -123
training
episopde 774 epsilon 0.01952590550501638 reward -123.0
Episode 775, Failed! Reward -200
training
episopde 775 epsilon 0.019428275977491298 reward -200.0
Episode 776, Better! Reward -118
training
episopde 776 epsilon 0.01933113459760384 reward -118.0
Episode 777, Better! Reward -120
training
episopde 777 epsilon 0.01923447892461582 reward -120.0
Episode 778, Failed! Reward

Episode 852, Failed! Reward -200
training
episopde 852 epsilon 0.01320722210464221 reward -200.0
Episode 853, Failed! Reward -200
training
episopde 853 epsilon 0.013141185994118999 reward -200.0
Episode 854, Failed! Reward -200
training
episopde 854 epsilon 0.013075480064148404 reward -200.0
Episode 855, Failed! Reward -200
training
episopde 855 epsilon 0.013010102663827662 reward -200.0
Episode 856, Failed! Reward -200
training
episopde 856 epsilon 0.012945052150508524 reward -200.0
Episode 857, Failed! Reward -200
training
episopde 857 epsilon 0.012880326889755982 reward -200.0
Episode 858, Better! Reward -130
training
episopde 858 epsilon 0.012815925255307201 reward -130.0
Episode 859, Better! Reward -119
training
episopde 859 epsilon 0.012751845629030666 reward -119.0
Episode 860, Better! Reward -121
training
episopde 860 epsilon 0.012688086400885512 reward -121.0
Episode 861, Better! Reward -125
training
episopde 861 epsilon 0.012624645968881083 reward -125.0
Episode 862, Failed! 

Episode 936, Failed! Reward -200
training
episopde 936 epsilon 0.008668625958465822 reward -200.0
Episode 937, Better! Reward -154
training
episopde 937 epsilon 0.008625282828673492 reward -154.0
Episode 938, Passed! Reward -94
training
episopde 938 epsilon 0.008582156414530124 reward -94.0
Episode 939, Better! Reward -119
training
episopde 939 epsilon 0.008539245632457474 reward -119.0
Episode 940, Better! Reward -120
training
episopde 940 epsilon 0.008496549404295187 reward -120.0
Episode 941, Better! Reward -122
training
episopde 941 epsilon 0.00845406665727371 reward -122.0
Episode 942, Better! Reward -115
training
episopde 942 epsilon 0.008411796323987343 reward -115.0
Episode 943, Better! Reward -114
training
episopde 943 epsilon 0.008369737342367406 reward -114.0
Episode 944, Better! Reward -155
training
episopde 944 epsilon 0.008327888655655568 reward -155.0
Episode 945, Failed! Reward -200
training
episopde 945 epsilon 0.00828624921237729 reward -200.0
Episode 946, Failed! Rew

In [11]:
Qvals[0],target_Qvals[0],Calc_Qvals[0],actions_ep[0]

(array([-2.7941396, -2.8588789, -2.531501 ], dtype=float32),
 array([-2.7941396, -2.8588789, -2.1366205], dtype=float32),
 -2.1366206407546997,
 2)

In [12]:
state,list(state)
MC.action_shape[0]
MC.action_shape[0]
np.random.randint(0,2)

1